In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
data=pd.read_csv('GEFCom2014N.csv').dropna()
data['load']=data['load']/10000
data['T']=data['T']/100
# Convert timestamp to datetime format
data['Datetime'] = data['Datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

# Set the datetime as index
data.set_index('Datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['load'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['load'].rolling(window=24).mean()
data['rolling_std'] = data['load'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

In [3]:
#CNN LeNet
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers

from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import mean_squared_error
# Prepare the train and test data
X_train, y_train = train_data.drop('load', axis=1), train_data['load']
X_test, y_test = test_data.drop('load', axis=1), test_data['load']


# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LeNet model
model = models.Sequential()
model.add(layers.Conv1D(6, kernel_size=5, activation='relu',  input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(16, kernel_size=5, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(120, activation='relu'))
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(1, activation='linear'))  # Output layer with linear activation for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

# Make predictions on new data
predictions = model.predict(X_test)
ypred=predictions
print('Predictions:', predictions)
ypred = ypred.flatten()
y_test=test_data['load']
y_lenet=ypred



Epoch 1/50
1972/1972 [==============================] - 5s 2ms/step - loss: 9.9120e-04
Epoch 2/50
1972/1972 [==============================] - 4s 2ms/step - loss: 1.4089e-04
Epoch 3/50
1972/1972 [==============================] - 4s 2ms/step - loss: 9.5958e-05
Epoch 4/50
1972/1972 [==============================] - 5s 2ms/step - loss: 7.3588e-05
Epoch 5/50
1972/1972 [==============================] - 5s 2ms/step - loss: 6.0134e-05
Epoch 6/50
1972/1972 [==============================] - 5s 2ms/step - loss: 5.0912e-05
Epoch 7/50
1972/1972 [==============================] - 5s 2ms/step - loss: 4.3445e-05
Epoch 8/50
1972/1972 [==============================] - 5s 2ms/step - loss: 3.8680e-05
Epoch 9/50
1972/1972 [==============================] - 5s 2ms/step - loss: 3.6204e-05
Epoch 10/50
1972/1972 [==============================] - 5s 2ms/step - loss: 3.2330e-05
Epoch 11/50
1972/1972 [==============================] - 5s 2ms/step - loss: 3.1063e-05
Epoch 12/50
1972/1972 [==================

In [4]:
#For MAE
mae = np.mean(np.abs(ypred - y_test))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test - ypred) / y_test)) * 100
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
mse = mean_squared_error(y_test, ypred) 
rmse = np.sqrt(mse) 
print('RMSE = {:.2f} (%)'.format(rmse))

MAE = 0.00 (%)
MAPE = 0.77 (%)
RMSE = 0.00 (%)
